In [ ]:
#| notest
#| eval: false
#| hide

import sys

sys.path.insert(0, './')
sys.path.insert(0, '../../')

import big_project_helper as bph

bph.display_project_contents()


llm_tools.py
import requests
import litellm
import json
from lisette import lite_mk_func

TOOLS = {}
TOOL_SCHEMAS = []


def get_tools():
    """Return available tool schemas."""
    return TOOL_SCHEMAS


def register_tool(func):
    """Register a function as a tool."""
    TOOLS[func.__name__] = func
    schema = lite_mk_func(func)
    # Remove old schema if exists
    TOOL_SCHEMAS[:] = [s for s in TOOL_SCHEMAS if s['function']['name'] != func.__name__]
    TOOL_SCHEMAS.append(schema)
    return func


@register_tool
def search_web(query: str, max_results: int = 10):
    """Search DuckDuckGo and return top results.
    
    Args:
        query: Search string.
        max_results: Maximum number of results to return.
        
    Returns:
        JSON string of results with title, url, snippet.
    """    
    from ddgs import DDGS
    
    results = DDGS().text(query, max_results=max_results)
    return str([{"title": r["title"], "url": r["href"], "snippet": r["body"]} 
            f

In [ ]:
#| default_exp config


In [ ]:
#| export

KERNEL_URL = 'http://localhost:8765'
MODEL = 'gpt-5.2'

PROMPT_SPLIT = '\n\n##### LLM Response: <!-- LLM -->\n\n'
AGENT_CODE_SPLIT = '\n\n##### Agent Code: <!-- AGENTCODE -->\n\n'
AGENT_CODE_OUTPUT_SPLIT = '\n\n##### Code Unit Test Output: <!-- UNITTEST -->\n\n'
AGENT_LLM_OUTPUT_SPLIT = '\n\n##### Agent Review: <!-- AGENTREVIEW -->\n\n'

UNIT_TEST_STR = """Review the results of the UNIT TEST.\n
Take into consideration: IS THE TOOL COMPLETING THE ORIGINAL GOAL?\n
Consider if anything about the UNIT TEST results shows a possible problem.\n
Note if the test doesn't have print statements, or isn't called ei: unit_test().\n
Make sure your explanation of any problems is clear.\n
This doesnt have to be scientifically perfect!( If it doesnt follow perfect code convention its ok ). \n
It just has to work, and match the users requirements.\n
Ultimately you must decide PASS or FAIL.\n
Explain thoroughly, but concisely!\n
"""

CODE_GENERATOR = """Generate code and a unit test.\n
Put the main function and unit test in code.\n
The UNIT TEST, must test the operation of the tool, by running it, and then querying the unreal scene to verify the results.\n
The UNIT TEST must use print(), NEVER USE console.log().\n
THE UNIT TEST FUNCTION MUST BE EXECTUTED ei: unit_test().\n 
DONT respond with words, only output raw executable CODE.\n
DONT use excessive try; excepts or asserts\n
Dont make Makrdown, Do NOT start with ```python !!\n
"""

CODE_IMPROVER = """Improve the code based on feedback.\n
Include the complete corrected code.\n
The UNIT TEST must use print(), NEVER USE console.log().\n
THE UNIT TEST FUNCTION MUST BE EXECTUTED ei: unit_test()\n
DONT respond with words, only output raw executable CODE.\n
DONT use excessive try; excepts or asserts\n
Dont make Makrdown, Do NOT start with ```python !! \n
"""

SYS_CODER = """You are a python coder, you are writing code for unreal engine 5.6.\n
You review the log of the goal, the previos code and the results of the unit test.\n
You take this into account as yout write new code\n
"""

SYS_REVIEW = """You are a code reviewer. Your job is to study the code and answer questions about it.
We are working in an unreal engine 5.6 environment\n"""

SYS_PROMPT = """You are an experiences python developer, with unreal engine 5.6.\n
Your job is to either review existing code, or write new code, or improve existing code\n

--- Use your privded tool ---
Do any preliminary websearch you need, check help urls\n
Our General Code Cycle is: \n
- generateCode\n
- unitTest   - This executes the code in unreal engine\n
while unitTest not passed:
    - improveCode\n
    - unitTest\n
        - If AN AttributeError: is found in unitTest, do a websearch, check urls\n
    - REPEAT!
"""

NOTEBOOK_SYS_PROMPT = """You are an AI assistant, your goal is to help the user build tools.\n
You're in a Jupyter Notebook, which is connected to Unreal Engine 5.6.\n
The users programming language of choice is python.\n
!!IMPORTANT!!: Always end your response with an explantion\n"""